In [1]:
import pandas as pd
import sqlite3
import sys

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

### Pull in zip_cbsa table

In [4]:
db = sqlite3.connect('../data/nppes_lite.sqlite')

query = """
SELECT * 
FROM zip_cbsa
"""
zip_cbsa = pd.read_sql(query,db)

db.close()

In [5]:
zip_cbsa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47424 entries, 0 to 47423
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   zip        47424 non-null  object 
 1   cbsa       47424 non-null  int64  
 2   res_ratio  47424 non-null  float64
 3   bus_ratio  47424 non-null  float64
 4   oth_ratio  47424 non-null  float64
 5   tot_ratio  47424 non-null  float64
dtypes: float64(4), int64(1), object(1)
memory usage: 2.2+ MB


In [6]:
#drop unecessary columns
zip_cbsa = zip_cbsa.drop(["res_ratio", "bus_ratio", "oth_ratio"], axis=1)

In [7]:
# sort records by zip and zip ratio
zip_cbsa = zip_cbsa.sort_values(['zip', 'tot_ratio'], 
                                ascending=[False, False])

In [8]:
#where a given zip code is in multiple CBSAs
#keep only the zip CBSA records with the greatest % of that zip
zip_cbsa = zip_cbsa.drop_duplicates(subset='zip', keep='first')

In [9]:
zip_cbsa.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39451 entries, 47423 to 0
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   zip        39451 non-null  object 
 1   cbsa       39451 non-null  int64  
 2   tot_ratio  39451 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.2+ MB


### Pull in nppes table

In [39]:
#query= """
#    SELECT sql 
#    FROM sqlite_master 
#    WHERE name = 'nppes'
#    """
#nppes_info.sql.unique()

In [10]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

query = """
SELECT 
    npi,
    entity_type_code,
    org_name,
    last_name,
    first_name,
    name_suffix,
    provider_credential,
    address_1,
    zip,
    taxonomy_code
FROM nppes
"""
nppes = pd.read_sql(query,db)

db.close()

In [11]:
#Random sample of 5 zip codes from table.
#Records will have either 5 or 9 digit zip codes.
nppes.zip.sample(5)

94809     378794352.0
5565          37208.0
115354    381194701.0
24970     384644455.0
81744     371294330.0
Name: zip, dtype: float64

In [12]:
#change column from float to string & keep only the first 5 characters
nppes['zip'] = nppes['zip'].astype(str).str[:5]

In [13]:
nppes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115486 entries, 0 to 115485
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   npi                  115486 non-null  int64  
 1   entity_type_code     115486 non-null  float64
 2   org_name             24501 non-null   object 
 3   last_name            90982 non-null   object 
 4   first_name           90985 non-null   object 
 5   name_suffix          2915 non-null    object 
 6   provider_credential  74398 non-null   object 
 7   address_1            115486 non-null  object 
 8   zip                  115486 non-null  object 
 9   taxonomy_code        115486 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 8.8+ MB


In [14]:
nppes.sample(2)

,npi,entity_type_code,org_name,last_name,first_name,name_suffix,provider_credential,address_1,zip,taxonomy_code
43196,1023253119,1.0,None,DOZIER,SELENA,None,M.D.,1412 E REELFOOT AVE,38261,207Q00000X
114495,1679736516,2.0,JACKSON PATHOLOGY GROUP A PROFESSIONAL CORPORA...,None,None,None,None,708 W FOREST AVE,38301,207ZP0102X


#### Drop old nash_nppes table

In [15]:
# create a database or connect to an existing one
db = sqlite3.connect('../data/nppes_lite.sqlite')


# if you need to rewrite the database...
cursor = db.cursor()

# Drop the table and return a line that says that it's gone
cursor.execute("DROP TABLE nash_nppes")

print("Table dropped...")

OperationalError: no such table: nash_nppes

In [16]:
db.close()

In [17]:
nppes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115486 entries, 0 to 115485
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   npi                  115486 non-null  int64  
 1   entity_type_code     115486 non-null  float64
 2   org_name             24501 non-null   object 
 3   last_name            90982 non-null   object 
 4   first_name           90985 non-null   object 
 5   name_suffix          2915 non-null    object 
 6   provider_credential  74398 non-null   object 
 7   address_1            115486 non-null  object 
 8   zip                  115486 non-null  object 
 9   taxonomy_code        115486 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 8.8+ MB


### Merge nppes and zip_cbsa dataframes, filter for Nashville CBSA

In [18]:
nppes_cbsa = nppes.merge(zip_cbsa, on="zip")

nppes_cbsa.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114907 entries, 0 to 114906
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   npi                  114907 non-null  int64  
 1   entity_type_code     114907 non-null  float64
 2   org_name             24481 non-null   object 
 3   last_name            90423 non-null   object 
 4   first_name           90426 non-null   object 
 5   name_suffix          2904 non-null    object 
 6   provider_credential  73963 non-null   object 
 7   address_1            114907 non-null  object 
 8   zip                  114907 non-null  object 
 9   taxonomy_code        114907 non-null  object 
 10  cbsa                 114907 non-null  int64  
 11  tot_ratio            114907 non-null  float64
dtypes: float64(2), int64(2), object(8)
memory usage: 11.4+ MB


In [19]:
#filter for Nashville CBSA code
nppes_cbsa[nppes_cbsa.cbsa==34980].zip.nunique()

110

In [20]:
nash_nppes=nppes_cbsa[nppes_cbsa.cbsa==34980]

### Pull in fields to merge selected fields from taxonomy table

In [21]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

query = """
SELECT code, grouping, classification, specialization 
FROM nucc_taxonomy
"""
taxonomy = pd.read_sql(query,db)

db.close()

In [22]:
taxonomy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 865 entries, 0 to 864
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   code            865 non-null    object
 1   grouping        865 non-null    object
 2   classification  865 non-null    object
 3   specialization  625 non-null    object
dtypes: object(4)
memory usage: 27.2+ KB


In [23]:
nash_nppes = nash_nppes.merge(taxonomy, left_on='taxonomy_code', right_on='code')

In [26]:
nash_nppes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38148 entries, 0 to 38147
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   npi                  38148 non-null  int64  
 1   entity_type_code     38148 non-null  float64
 2   org_name             7466 non-null   object 
 3   last_name            30681 non-null  object 
 4   first_name           30682 non-null  object 
 5   name_suffix          895 non-null    object 
 6   provider_credential  25175 non-null  object 
 7   address_1            38148 non-null  object 
 8   zip                  38148 non-null  object 
 9   taxonomy_code        38148 non-null  object 
 10  grouping             38148 non-null  object 
 11  classification       38148 non-null  object 
 12  specialization       15120 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 4.1+ MB


In [25]:
nash_nppes = nash_nppes.drop(columns=['code','cbsa','tot_ratio'])

#### Save nash_nppes as table in database

In [27]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #open connection

nash_nppes.to_sql('nash_nppes', db, if_exists = 'append', index = False) #save nashville CBSA nppes

db.close() #close connection

#### Verify table data

In [28]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

query= """
    SELECT *
    FROM nash_nppes
    """
nash_nppes = pd.read_sql(query,db)

db.close

<function Connection.close>

In [29]:
nash_nppes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38148 entries, 0 to 38147
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   npi                  38148 non-null  int64  
 1   entity_type_code     38148 non-null  float64
 2   org_name             7466 non-null   object 
 3   last_name            30681 non-null  object 
 4   first_name           30682 non-null  object 
 5   name_suffix          895 non-null    object 
 6   provider_credential  25175 non-null  object 
 7   address_1            38148 non-null  object 
 8   zip                  38148 non-null  object 
 9   taxonomy_code        38148 non-null  object 
 10  grouping             38148 non-null  object 
 11  classification       38148 non-null  object 
 12  specialization       15120 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 3.8+ MB


#### Check memory usage of zip codes as float/int/str

https://stackoverflow.com/questions/20970279/how-to-do-a-left-right-and-mid-of-a-string-in-a-pandas-dataframe/20970328

https://towardsdatascience.com/the-strange-size-of-python-objects-in-memory-ce87bdfbb97f

https://www.geeksforgeeks.org/how-to-find-size-of-an-object-in-python/

https://sdsawtelle.github.io/blog/output/large-data-files-pandas-sqlite.html

In [48]:
nppes.memory_usage()

Index                     128
npi                    923888
entity_type_code       923888
org_name               923888
last_name              923888
first_name             923888
middle_name            923888
name_prefix            923888
name_suffix            923888
provider_credential    923888
address_1              923888
address_2              923888
city                   923888
state                  923888
zip                    923888
taxonomy_code          923888
dtype: int64

In [16]:
sys.getsizeof(nppes)

72233395

In [24]:
print(sys.getsizeof(nppes['zip'])) #zip column as float64

print(sys.getsizeof(nppes['zip'].astype(str))) #zip column as string

print(sys.getsizeof(nppes['zip'].astype(str).str[:5])) #just first 5 characters of zip col as str

924032
7805687


#### Isolate Nashville-Davidson CBSA zip codes
CBSA id is 34980

In [ ]:
nash_zip_cbsa = zip_cbsa.copy()
nash_zip_cbsa = nash_zip_cbsa[nash_zip_cbsa.cbsa==34980]
nash_zip_cbsa.info()

#### Alternately, do datatype conversion & string manipulation in SQL query

In [98]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

query= """
    SELECT
        npi,
        entity_type_code,
        org_name,
        last_name,
        first_name,
        middle_name,
        name_prefix,
        name_suffix,
        provider_credential,
        address_1,
        address_2,
        city,
        state,
        SUBSTR(nppes.zip, 1, 5) AS zip,
        taxonomy_code
    FROM nppes
    """
nppes_zip5 = pd.read_sql(query,db)

db.close


<function Connection.close>

In [99]:
nppes_zip5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115486 entries, 0 to 115485
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   npi                  115486 non-null  int64  
 1   entity_type_code     115486 non-null  float64
 2   org_name             24501 non-null   object 
 3   last_name            90982 non-null   object 
 4   first_name           90985 non-null   object 
 5   middle_name          62237 non-null   object 
 6   name_prefix          34723 non-null   object 
 7   name_suffix          2915 non-null    object 
 8   provider_credential  74398 non-null   object 
 9   address_1            115486 non-null  object 
 10  address_2            26571 non-null   object 
 11  city                 115486 non-null  object 
 12  state                115486 non-null  object 
 13  zip                  115486 non-null  object 
 14  taxonomy_code        115486 non-null  object 
dtypes: float64(1), in

In [105]:
#check random samples to make sure the zipcode is formatted correctly
nppes_zip5.sample()

,npi,entity_type_code,org_name,last_name,first_name,middle_name,name_prefix,name_suffix,provider_credential,address_1,address_2,city,state,zip,taxonomy_code
91681,1790339075,1.0,None,BARNES,APRIL,None,None,None,None,2250 WILMA RUDOLPH BLVD STE F259,None,CLARKSVILLE,TN,37040,106S00000X


In [101]:
#merge dataframes
nppes_nash = nppes_zip5.merge(nash_zip_cbsa, on="zip")

In [102]:
nppes_nash.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38148 entries, 0 to 38147
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   npi                  38148 non-null  int64  
 1   entity_type_code     38148 non-null  float64
 2   org_name             7466 non-null   object 
 3   last_name            30681 non-null  object 
 4   first_name           30682 non-null  object 
 5   middle_name          19790 non-null  object 
 6   name_prefix          11470 non-null  object 
 7   name_suffix          895 non-null    object 
 8   provider_credential  25175 non-null  object 
 9   address_1            38148 non-null  object 
 10  address_2            9877 non-null   object 
 11  city                 38148 non-null  object 
 12  state                38148 non-null  object 
 13  zip                  38148 non-null  object 
 14  taxonomy_code        38148 non-null  object 
 15  cbsa                 38148 non-null 

In [108]:
nppes_nash.zip.nunique()

110

In [103]:
nppes_all = nppes_zip5.merge(zip_cbsa, on="zip")

In [104]:
nppes_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114907 entries, 0 to 114906
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   npi                  114907 non-null  int64  
 1   entity_type_code     114907 non-null  float64
 2   org_name             24481 non-null   object 
 3   last_name            90423 non-null   object 
 4   first_name           90426 non-null   object 
 5   middle_name          61882 non-null   object 
 6   name_prefix          34498 non-null   object 
 7   name_suffix          2904 non-null    object 
 8   provider_credential  73963 non-null   object 
 9   address_1            114907 non-null  object 
 10  address_2            26314 non-null   object 
 11  city                 114907 non-null  object 
 12  state                114907 non-null  object 
 13  zip                  114907 non-null  object 
 14  taxonomy_code        114907 non-null  object 
 15  cbsa             

In [109]:
nppes_all[nppes_all.cbsa==34980].zip.nunique()

110

In [ ]:
1841293891